# Langchain

## Acknowledgement

This notebook is based off: https://github.com/gkamradt/langchain-tutorials

#### SageMaker Studio Notebook
This notebook has been tested in us-east-1 with **Data Science 3.0** kernel

----

In [ ]:
!pip install --upgrade pip
!pip install transformers faiss-gpu --quiet
!pip install bs4 --quiet

In [ ]:
#!pip install upgrade sagemaker
!pip install langchain

In [ ]:
import langchain
import requests
import json

print(langchain.__version__)
# assert int(langchain.__version__.split(".")[-1]) >= 194

# Model Deployment

### Deploy from SageMaker JumpStart- We will deploy textembedding-gpt-6b-fp16 in this lab.

#### <mark> Step 1</mark>, we go to SageMaker Jumpstart and deploy one of the gpt-j embedding models- we will use this model further down to generate embeddings.

- <mark> textembedding-gpt-j-6b-fp16 (ml.g5.4xlarge)</mark> #this notebook deploys and uses this one.
or
- textembedding-gpt-j-6b (ml.g5.12xlarge)  



In [ ]:
_MODEL_CONFIG_ = {
    
    "textembedding-model": {
        "aws_region": "us-east-1",
        "endpoint_name": "jumpstart-dft-textembedding-gpt-j-6b-fp16-1",
    },
    
    "llm-model" : {
        "aws_region": "us-east-1",
        "endpoint_name": "demo-Falcon40B-Endpoint",
        "api_url": "https://t7zr78elmj.execute-api.us-east-1.amazonaws.com/prod/falcon",
        "headers":{
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': 'xxx'  #insert the authentication code
}

    },
}


#### Falcon 40B model has been already delpoyed on a ml.g5.24x instance and the url where you can invoke and interact with the model is provided in the following together with and the authentication passed on as part of the message. see the header that contains authentication inforamtion in our llm model configuration we defined above ( a copy in the following)    
<code>
        "llm-model" : {
        "aws_region": "us-east-1",
        "endpoint_name": "demo-Falcon40B-Endpoint",
        "api_url": "https://t7zr78elmj.execute-api.us-east-1.amazonaws.com/prod/falcon",
        "headers":{
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': 'xxxx'
    }
</code>

# Langchain

## Schema
This section covers the basic data types and schemas used in Langchain.
There are few data type/ schema supported by langchain- we will explore text and documents in the following section.
- text
- document
- examples
- chat messages (covered in optional section)


## Text

You'll be working with simple strings - spoiler alert: that'll soon grow in complexity! ;) 

In [ ]:
my_text = "What day comes after Friday?"

## Documents

In [ ]:
from langchain.schema import Document

Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "The LangChain Papers",
             'my_document_create_time' : 1680013019
         })

# Models
There are three type of models in Lanchain- Language Model, Chat model and text Embedding Model- in the following we will interact with text embedding and language


## Language Models
Lets invoke our Flacon model using a simple string prompt. 

In [ ]:
from langchain.llms import AmazonAPIGateway

parameters ={
        "max_new_tokens": 100,
        "num_return_sequences": 1,
        "top_k": 50,
        "top_p": 0.95,
        "do_sample": False,
        "return_full_text": False,
        "temperature": 0.2
}


sm_llm_falcon_instruct = AmazonAPIGateway(
    api_url=_MODEL_CONFIG_["llm-model"]["api_url"], 
    model_kwargs=parameters,
    headers=_MODEL_CONFIG_["llm-model"]["headers"],
)

In [ ]:
sm_llm_falcon_instruct("what is Amazon SageMaker?")

## Text Embedding Models

We need to Wrap up our SageMaker endpoints for embedding model into <mark>langchain.embeddings.SagemakerEndpointEmbeddings</mark>. That requires a small overwritten of SagemakerEndpointEmbeddings class to make it compatible with SageMaker embedding mdoel. Same applies to use the SageMaker endpoint for Language models- In the above you notice we are using the APIGateway class for the Langchain

In [ ]:
from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json


class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: List[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"text_inputs": inputs, **model_kwargs})
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]

emb_content_handler = ContentHandler()


embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=_MODEL_CONFIG_["textembedding-model"]["endpoint_name"],
    region_name=_MODEL_CONFIG_["textembedding-model"]["aws_region"],
    content_handler=emb_content_handler
)

In [ ]:
text = "Hi! It's time for the beach"

text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

In [ ]:
doc_embedding = embeddings.embed_documents([text])
print (f"Your embedding is length {len(doc_embedding[0])}")
print (f"Here's a sample: {doc_embedding[0][:5]}...")

## Prompt
The new way of programming models is through prompts. A "prompt" refers to the input to the model. This input is rarely hard coded, but rather is often constructed from multiple components.
This is an example of a simple text prompt, passed to our model to get some response.

In [ ]:
prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

sm_llm_falcon_instruct(prompt)

## Prompt Template
This is an example of a simple prompt template, passed on to our model to get some response.
A PromptValue is what is eventually passed to the model. Most of the time, this value is not hardcoded but is rather dynamically created based on a combination of user input, other non-static information (often coming from multiple sources), and a fixed template string. We call the object responsible for creating the PromptValue a PromptTemplate. This object exposes a method for taking in input variables and returning a PromptValue.

In [ ]:
from langchain import PromptTemplate

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Melbourne')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {sm_llm_falcon_instruct(final_prompt)}")

# Chain
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case. The most commonly used type of chain is an LLMChain, which combines a PromptTemplate, a Model, and Guardrails to take user input, format it accordingly, pass it to the model and get a response, and then validate and fix (if necessary) the model output.

The following shows few examples of chains that combines a promot tempalte and a LLM.

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

In [ ]:
parameters = {
    "max_new_tokens": 1500,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": False,
    "temperature": 1,
    "seed": 123
}

sm_llm_falcon_instruct.model_kwargs = parameters

### LLMChain: 
LLM Chain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format. This is a simple LLMchain that consists of a prompt template and a language model

In [ ]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=sm_llm_falcon_instruct, prompt=prompt_template)
output=location_chain.run("Melbourne")
output

In [ ]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=sm_llm_falcon_instruct, prompt=prompt_template)

### Now an example of chain, that sequentally chain two LLMs together with the user prompt

In [ ]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)
review = overall_chain.run("Melbourne")

### Summarization Chain
An example of Summarization chain is created in the following.
- first we use documents as our input ( other input format beyond the simple text we discussed above)- we use document_loader class to load the document
- second, we split the text inot chunks before feeding it into the model


In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('data/Amazon_SageMaker_FAQs.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=5)

# Split your docs into texts
texts = text_splitter.split_documents(documents)
len(texts)

In [ ]:
parameters = {
    "max_new_tokens": 100,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": False,
    "temperature": 1,
}

sm_llm_falcon_instruct.model_kwargs = parameters

In [ ]:
# There is a lot of complexity hidden in this one line. I encourage you to check out the video above for more detail
# chain = load_summarize_chain(sm_llm, chain_type="map_reduce", verbose=True)
chain = load_summarize_chain(sm_llm_falcon_instruct, chain_type="map_reduce", verbose=True)
chain.run(texts[:2])

#### There is a lot more to learn and discover with Langchains, inclusing output parsers, agents, tools, growing rapidly. You can learn more about Langchain capabilities here https://github.com/gkamradt/langchain-tutorials

## Clean up

- Delete deployed LLM endpoint